In [1]:
import torch
from torch import optim,nn
from torch.autograd import Variable
import numpy as np
import time
from models.data_loader import DataLoader
from models.retain_bidirectional import RETAIN

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [3]:
# hyperparameters
epochs = 30
batch_size = 50
max_seq_length = 300
min_seq_length = 5
num_classes = 268
emb_size = 128
hid_size = 128
lr = 0.001
cuda_flag = True

# data loader
D = DataLoader(batch_size=batch_size,
   data_dir='data/batches/',
    mode='train', max_seq_length=max_seq_length, min_seq_length=min_seq_length)

# import model and optimization settings
model = RETAIN(emb_size,hid_size,num_classes,cuda_flag)
model.release = True

criterion = nn.CrossEntropyLoss()
cnt = 0
if cuda_flag:
    model.cuda()
    criterion.cuda()

In [4]:
# train model
lr_list = [0.001, 0.0003, 0.0001, 0.00003, 0.00001, 0.000003, 0.000001]
lr_counter = 0
lr = lr_list[lr_counter]
opt = optim.Adam(model.parameters(), lr=lr)
loss_list = []
loss_mean = 0.0
file_cnt = 0
cnt = 0
loss_dict = dict()

In [ ]:
len_train = len(D.train_list)
while file_cnt<(epochs*len(D.train_list)):
    idx = file_cnt%(len_train)
    file = D.train_list[idx]
    if file not in loss_dict:
        loss_dict[file] = []
    print("Epoch %d [%d, %d/%d] - opening file %s" %(((file_cnt+1)/len_train), file_cnt, idx, len_train, D.train_list[idx]))
    file_num = int(file.split('_')[1].split('.')[0])
    D.batch_size = int(20000/file_num)
    D.load_batch_file(file)
    loss_list = []
    for i in range(D.batch_count):
        cnt+=1
        input_list, targets = D.get_batch()
        start = time.time()
        inputs = model.list_to_tensor(input_list)
        outputs = model(inputs)
        targets = Variable(torch.LongTensor(targets)[:,-1]) # to only use last of each sequence
#             targets = Variable(torch.LongTensor(targets)).view(len(inputs),-1)[:,-1] # to only use last of each sequence
        if cuda_flag:
            targets = targets.cuda()
        loss = criterion(outputs.view(-1,num_classes),targets)
        loss_list.append(loss.data[0])
        if cnt%10==0:
            print('[%d] %1.3f' %(cnt,loss.data[0]))
        if cnt%500==0:
            print("Saving model at %dth step" %cnt)
            torch.save(model,'data/saved_weights/retain_bi_%d.pth'%(cnt))
            # create CPU version
            model2 = RETAIN(emb_size,hid_size,num_classes,False)
            if cuda_flag:
                model.cpu()
            model2.load_state_dict(model.state_dict())
            torch.save(model2,'data/saved_weights/retain_bi_%d_cpu.pth'%(cnt))
            if cuda_flag:
                model.cuda()
            print("Saving at %dth step"%cnt)
        # manual loss changes
        if cnt==100:
            lr_counter+=1
            lr = lr_list[lr_counter]
            opt = optim.Adam(model.parameters(),lr=lr)
        if cnt==500:
            lr_counter+=1
            lr = lr_list[lr_counter]
            opt = optim.Adam(model.parameters(),lr=lr)        
        if loss.data[0]>10:
            import sys
            sys.exit()
#             print(loss.data[0])
        loss.backward()
        opt.step()
    print("Loss: %1.3f" %np.mean(loss_list))
    loss_dict[file].append(loss.data[0])
    file_cnt+=1

Epoch 1 [727, 317/410] - opening file 2015_49.pckl


/home/mjc/github/EHRVis/models/retain_bidirectional.py:35: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  outputs1 = self.RNN1(embedded) # [b x seq x 128*2]


Loss: 2.985
Epoch 1 [728, 318/410] - opening file 2014_232.pckl
Loss: 3.396
Epoch 1 [729, 319/410] - opening file 2015_112.pckl
Loss: 3.010
Epoch 1 [730, 320/410] - opening file 2015_288.pckl
[2510] 0.460
Loss: 0.460
Epoch 1 [731, 321/410] - opening file 2015_74.pckl
Loss: 2.919
Epoch 1 [732, 322/410] - opening file 2014_14.pckl
[2520] 3.038
[2530] 3.051
Loss: 3.068
Epoch 1 [733, 323/410] - opening file 2014_101.pckl
[2540] 2.545
Loss: 2.756
Epoch 1 [734, 324/410] - opening file 2015_7.pckl
[2550] 3.546
[2560] 3.388
Loss: 3.526
Epoch 1 [735, 325/410] - opening file 2015_154.pckl
Loss: 1.666
Epoch 1 [736, 326/410] - opening file 2014_151.pckl
Loss: 1.967
Epoch 1 [737, 327/410] - opening file 2014_5.pckl
[2570] 3.773
[2580] 3.709
Loss: 3.777
Epoch 1 [738, 328/410] - opening file 2014_256.pckl
Loss: 1.316
Epoch 1 [739, 329/410] - opening file 2015_237.pckl
Loss: 2.427
Epoch 1 [740, 330/410] - opening file 2015_123.pckl
Loss: 3.090
Epoch 1 [741, 331/410] - opening file 2014_190.pckl
Loss: 

In [8]:
# load new model
model = torch.load('/home/mjc/github/EHRVis/data/saved_weights/retain_bi_2500.pth')
file_cnt = 727
cnt = 2500

In [9]:
# lower learning rate
lr_counter+=1
lr = lr_list[lr_counter]
opt = optim.Adam(model.parameters(), lr=lr)